In [1]:
import pandas as pd
import sys
from datetime import datetime
from pycti import OpenCTIApiClient
from dateutil.parser import parse
import pandas as pd
import os
import time
import random
from shutil import copyfile

In [24]:
class anamoly_detection_and_mapping_with_ip_as_indicator():
    
    def __init__(self,log_file='data/weblog.log',api_url = "https://demo.opencti.io",api_token = "2b4f29e3-5ea8-4890-8cf5-a76f61f1e2b2"):
        
        self.log_file = log_file
        self.connection = OpenCTIApiClient(api_url,api_token)
        
    def anamoly_detection(self):
        
        df=pd.read_csv(self.log_file,header=None,delimiter=' ')
        df[2]=df[2]+df[3]+df[4]
        df.drop(columns=[3,4],inplace=True)
        df.columns=['indicator','Time','URL','Status']
        all_indicators=pd.read_csv('data/all_indicators.csv') # all_indicators_DB
        df['Time']=df['Time'].str.replace('[','').replace(']','')#removing unwanted in time columns
        df['Time']=list(map(lambda x:datetime.strptime(x, '%d/%b/%Y:%H:%M:%S'),list(df['Time']))) # converting to timestamp
        print('\t\t________summary report__________\n')
        
        print('Shape of the log Data\t\t',df.shape[0],'X',df.shape[1]) #print shape of log data
        print('Data Headers\t\t\t',list(df.columns))# printing availbale columns
        print('Period\t\t\t\t',df['Time'].min(),'to',df['Time'].max()) #printing the period
        print('Duration\t\t\t',df['Time'].max()-df['Time'].min()) #printing the duration
        print('Number of unique Indicators\t',len(df['indicator'].unique()))#printing  the  number of unique indicators
        print('unique status codes\t\t',df['Status'].unique(),'\n')#printing  the  number of unique status code
        unique_indicator=df['indicator'].unique()
        siem_db=['70.32.39.219','91.194.254.94','128.199.199.187','69.12.73.174','94.237.37.28'] #considering as SIEM DB

        print('\t\t>>>>>>>>>>finding anamolies in logs<<<<<<<<<<<\n')
        time.sleep(random.randint(4, 6))
        anamolies=dict()
        for indicator in unique_indicator:
            result=all_indicators[all_indicators['observable_value']==indicator] #checking in rhymtip platform for IOC
            if len(result)>0:
                anamolies[indicator]=list(result['threat_actor'])[0] # if available send it to anamolies

        already_available=0
        not_availbale=0
        already_found_anamoly=[]
        not_found_anamoly=[]
        for indicator in anamolies:
            if indicator in siem_db: #anamoly foundby SIEM
                already_available+=1
                already_found_anamoly.append(indicator)
            else:
                not_availbale+=1# anamoly foundby Rhymtip
                not_found_anamoly.append(indicator)
        print('Total number of anomalies-------------',already_available+not_availbale)
        print('Total number of Anomaly detected and rejected by SIEM-------------',already_available,'\n')
        print(all_indicators.set_index('observable_value').loc[already_found_anamoly],'\n')
        print('Total number of Anomaly not detected by SIEM-------------',not_availbale,'\n')
        anamoly_groups=all_indicators.set_index('observable_value').loc[not_found_anamoly]
        print(anamoly_groups)
        
        return anamoly_groups #returning anamoly groups
    
    def create_report(self): #creating a report for publishing anamolies
    
        report_data={'name':'Log_analysis_report_'+datetime.today().strftime("%Y-%m-%dT%H:%M:%S+00:00"), #log analysis name
                    'description':'Log analysis and anamoly detection '+str(self.path.split('\\')[-1]),#log file name
                    'object_status':0, # 0,1,2 0-New,1-,2-, to mention the log report status
                    'source_confidence_level':1,#1,2,3 for confidence on the report
                    'created_by_ref':'Rhymtechnologies',#published by rhym technologies
                    'marking_definitions':None} #default it is white
        
        report_details=pd.DataFrame([report_data.values()],columns=report_data.keys()) #creating  a report df
        for index,row in report_details.iterrows(): #iterating report_details
            # cheking for Rhymtechnologies organiation if not it creates a identity in the name of rhymtechnologies
            report_by = self.connection.identity.list(filters=[{'key':'name','values': row['created_by_ref'] }])
           
            if len(report_by) == 0 : 

                report_by = organization = self.connection.identity.create(
                                            type='Organization',
                                            name="Rhymtechnologies",
                                            alias=["Rhymtech"],
                                            description="A young and innovative company formed with a vision to create world leading Cyber Security products and services. The promoters are a group of highly successful professionals with diverse backgrounds across IT Delivery, Sales and Consulting, Cyber Security, Artificial Intelligence, Machine Learning and Data Sciences. Our group of consultants led by some of the best minds in the industry have successfully provided strategic and value-add services for multiple large organizations globally.",
                                            )
            else:
                report_by = report_by[0]
            
            #creating a report to publish logs
            report = self.connection.report.create(
                                                            name=row['name'],
                                                            description= row['description'],
                                                            report_class="Internal Report",
                                                            published=datetime.today().strftime("%Y-%m-%dT%H:%M:%S+00:00"),
                                                            createdByRef= report_by['id'],
                                                            object_status = row['object_status'],
                                                            source_confidence_level = row ['source_confidence_level'],
                                                            update = True,
                                                            )
            print("one_completed")
            
        return report
    
    def create_indicator_and_add_to_report(self,report_id,anamoly_groups):#relating indicators to our report
        indicator_dict=dict()
        for index,row in anamoly_groups.iterrows():
                observables = self.connection.stix_observable.create(type="IPv4-Addr", observable_value=index, createIndicator=True)
                indicator_id=observables["indicators"][0]['id']
                self.connection.report.add_stix_entity(id=report_id,entity_id =indicator_id)
                indicator_dict[indicator_id]=row['threat_actor']
        return indicator_dict
    
    def threat_group_mapping(self,threat_group_name,report_id,indicators):#threat group mapping
        #fetching of intrusion set details
        intrusion_set = self.connection.intrusion_set.read(filters=[{"key": "name", "values": [threat_group_name]}])
        stix_relations = self.connection.stix_relation.list(fromId=intrusion_set["id"], inferred=True)
        all_details=[]
        for relation in stix_relations:
            details=[relation['from']['id'],relation['id'],relation['to']['id'],relation['from']['name'],relation['relationship_type'],relation['to']['name'],relation['to']['entity_type']]
            all_details.append(details)
        df_relations=pd.DataFrame(all_details,columns=['from_id','relation_id','to_id','from_name','relation','to_name','to_type'])

        all_id=list(set(df_relations['from_id']))
        all_id.extend(set(df_relations['to_id']))
        all_id.extend(set(df_relations['relation_id']))

        #adding entities to report
        result =list(map(lambda x:self.connection.report.add_stix_entity(id=report_id,entity_id =x),all_id))

        group_id=list(set(df_relations['from_id']))[0]
        group_id_name=self.connection.get_stix_domain_entity (group_id)['name'].lower()
        for indicator_id,group_name in indicators.items():
            if group_name.lower()==group_id_name:
                relation = self.connection.stix_relation.create(
                        fromType= self.connection.get_stix_domain_entity (group_id)['entity_type'],
                        fromId=group_id,
                        relationship_type ='indicates',
                        toType=self.connection.get_stix_domain_entity (indicator_id)['entity_type'],
                        toId= indicator_id,
                        )
            #connecting relation of indicator to stix entity
            self.connection.report.add_stix_entity(id=report_id,entity_id =relation['id'])


        return df_relations
    
    
if __name__ == '__main__':
    rhym=anamoly_detection_and_mapping_with_ip_as_indicator()
    a=rhym.anamoly_detection()
#     r=rhym.create_report()
#     i=rhym.create_indicator_and_add_to_report(anamoly_groups=a,report_id=r['id'])
#     all_relations=rhym.threat_group_mapping(threat_group_name='CARBANAK',report_id=r['id'],indicators=i)
#     copyfile(r'D:\MY STUFF\rhym\use_cases\Tip_Report_template.pdf','data'+os.sep+'Log_analysis_report_'+datetime.today().strftime("%Y-%m-%dT")+r['id']+'.pdf')
#     print('please go through this link for network graph https://demo.opencti.io/dashboard/reports/all/'+r['id']+'/knowledge')
    

INFO:root:Listing Threat-Actors with filters null.


		________summary report__________

Shape of the log Data		 15789 X 4
Data Headers			 ['indicator', 'Time', 'URL', 'Status']
Period				 2019-11-07 23:59:19 to 2020-03-02 15:47:46
Duration			 115 days 15:48:27
Number of unique Indicators	 13
unique status codes		 [200 302 304 206 404 401] 

		>>>>>>>>>>finding anamolies in logs<<<<<<<<<<<

Total number of anomalies------------- 8
Total number of Anomaly detected and rejected by SIEM------------- 5 

                 type threat_actor
observable_value                  
70.32.39.219       ip        turla
91.194.254.94      ip     carbanak
69.12.73.174       ip        apt28
94.237.37.28       ip        apt28
128.199.199.187    ip        apt28 

Total number of Anomaly not detected by SIEM------------- 3 

                 type threat_actor
observable_value                  
81.17.30.29        ip        apt28
92.114.92.102      ip        apt28
10.74.5.100        ip     carbanak


In [4]:
print('please go through this link for network graph https://demo.opencti.io/dashboard/reports/all/'+r['id']+'/knowledge')

please go through this link for network graph https://demo.opencti.io/dashboard/reports/all/8d83a54b-b35a-4d9d-9a9d-ea20ef070243/knowledge


In [82]:
rhym=anamoly_detection_and_mapping_with_ip_as_indicator()
a=rhym.anamoly_detection()
r=rhym.create_report()
i=rhym.create_indicator_and_add_to_report(anamoly_groups=a,report_id=r['id'])
all_relations=rhym.threat_group_mapping(threat_group_name='apt28',report_id=r['id'],indicators=i)

INFO:root:Listing Threat-Actors with filters null.


		________summary report__________

Shape of the log Data		 15789 X 4
Data Headers			 ['indicator', 'Time', 'URL', 'Status']
Period				 2017-11-07 23:59:19 to 2018-03-02 15:47:46
Duration			 114 days 15:48:27
Number of unique Indicators	 13
unique status codes		 [200 302 304 206 404 401] 

		>>>>>>>>>>finding anamolies in logs<<<<<<<<<<<



INFO:root:Listing Identities with filters [{"key": "name", "values": "Rhymtechnologies"}].


Total number of anomalies------------- 8
Total number of Anomaly detected and rejected by SIEM------------- 5 

                 type threat_actor
observable_value                  
70.32.39.219       ip        turla
91.194.254.94      ip     carbanak
69.12.73.174       ip        apt28
94.237.37.28       ip        apt28
128.199.199.187    ip        apt28 

Total number of Anomaly not detected by SIEM------------- 3 

                 type threat_actor
observable_value                  
81.17.30.29        ip        apt28
92.114.92.102      ip        apt28
10.74.5.100        ip     carbanak


INFO:root:Listing Reports with filters [{"key": "name", "values": ["Log_analysis_report_2020-05-06T18:53:04+00:00"]}, {"key": "published_day", "values": ["2020-05-06"]}].
INFO:root:Creating Report {Log_analysis_report_2020-05-06T18:53:04+00:00}.
INFO:root:Listing StixObservables with filters [{"key": "observable_value", "values": ["81.17.30.29"]}].


...

INFO:root:Checking Stix-Entity {a85adccc-6446-473d-a647-36c2651a30da} in Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Adding Stix-Entity {a85adccc-6446-473d-a647-36c2651a30da} to Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Checking Stix-Entity {5333a1d7-0dd1-4f84-b9c1-ef74c691d125} in Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Adding Stix-Entity {5333a1d7-0dd1-4f84-b9c1-ef74c691d125} to Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Checking Stix-Entity {b88e8339-ae71-41ec-8db7-3a9a368aea4b} in Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Adding Stix-Entity {b88e8339-ae71-41ec-8db7-3a9a368aea4b} to Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Checking Stix-Entity {778a6f6d-45b3-498d-89b9-da9b3edda634} in Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Adding Stix-Entity {778a6f6d-45b3-498d-89b9-da9b3edda634} to Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Checking Stix-Entity {b6440a02-71eb-4720-b083-

INFO:root:Checking Stix-Entity {9e3fdf90-e7ca-451e-985e-f8df772761aa} in Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Adding Stix-Entity {9e3fdf90-e7ca-451e-985e-f8df772761aa} to Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Checking Stix-Entity {e36bb325-4e1a-46fc-8c1b-6c40233ddcaa} in Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Adding Stix-Entity {e36bb325-4e1a-46fc-8c1b-6c40233ddcaa} to Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Checking Stix-Entity {8773cc88-d09c-453c-8129-8d9bb142e3c8} in Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Adding Stix-Entity {8773cc88-d09c-453c-8129-8d9bb142e3c8} to Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Checking Stix-Entity {91686c89-1a6b-4027-8f62-4299845d1243} in Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Adding Stix-Entity {91686c89-1a6b-4027-8f62-4299845d1243} to Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Checking Stix-Entity {53c54c53-27a3-4489-bd64-

INFO:root:Checking Stix-Entity {88df96a3-764a-4629-9913-6c0bea4d2fb8} in Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Adding Stix-Entity {88df96a3-764a-4629-9913-6c0bea4d2fb8} to Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Checking Stix-Entity {de3feeb9-979a-4bcf-b60c-1a2186ad6129} in Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Adding Stix-Entity {de3feeb9-979a-4bcf-b60c-1a2186ad6129} to Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Checking Stix-Entity {c2ec38cf-312d-4cfa-8c71-6dbab35fd838} in Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Adding Stix-Entity {c2ec38cf-312d-4cfa-8c71-6dbab35fd838} to Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Checking Stix-Entity {9ccbbdc4-192e-40f1-9dd9-ca8218332a80} in Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Adding Stix-Entity {9ccbbdc4-192e-40f1-9dd9-ca8218332a80} to Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Checking Stix-Entity {86c0982a-2258-47a5-8097-

INFO:root:Checking Stix-Entity {12f0788a-5cb5-4917-87d3-e507f7a4c38a} in Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Adding Stix-Entity {12f0788a-5cb5-4917-87d3-e507f7a4c38a} to Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Checking Stix-Entity {7a8cb537-d116-4d26-a505-4232c735d36a} in Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Adding Stix-Entity {7a8cb537-d116-4d26-a505-4232c735d36a} to Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Checking Stix-Entity {646f4e16-2149-453b-8739-f482c59ca54e} in Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Adding Stix-Entity {646f4e16-2149-453b-8739-f482c59ca54e} to Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Checking Stix-Entity {2973de48-0be6-4a75-b81b-9eae3343d129} in Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Adding Stix-Entity {2973de48-0be6-4a75-b81b-9eae3343d129} to Report {4baf9ef7-c186-47ab-b35b-550496b59fef}
INFO:root:Checking Stix-Entity {ccf1e4dc-eae3-40c7-8474-

TypeError: 'NoneType' object is not subscriptable

In [25]:
df=pd.read_csv(r"D:\MY STUFF\rhym\use_cases\data\weblog.csv")
df['Time']=df['Time'].str.replace('[','').replace(']','')#removing unwanted in time columns
df['Time']=list(map(lambda x:datetime.strptime(x, '%d/%b/%Y:%H:%M:%S'),list(df['Time'])))

In [26]:
def finding_count_type_of_request(df):
    GET=0
    POST=0
    PUT=0
    others=0
    for url in df['URL']:
        if 'GET' in url:
            GET+=1
        elif 'POST' in url:
            POST+=1
        elif 'PUT' in url:
            PUT+=1
        else:
            others+=1
    return GET,POST,PUT,others        

In [27]:
initial=True
unique_indicators=df['indicator'].unique()
for ind in unique_indicators:
    data=df[df['indicator']==ind]
    size=len(data)
    get,post,put,others=finding_count_type_of_request(data)
    result=pd.DataFrame({'indicator':[ind],'size':[size],'GET':[get],'POST':[post],'PUT':[put],'OTHERS':[others]})
    if initial:
        all_result=result
        initial=False
    else:
        all_result=all_result.append(result)
all_result.to_csv('data/log_analysis_report.csv',index=None)

In [53]:
df_ind=df.set_index('indicator')

In [72]:
df_ind[df_ind.index=='69.12.73.174']

,Time,URL,Status
indicator,,,
69.12.73.174,2018-01-29 20:29:41,GET /js/vendor/modernizr-2.8.3.min.js HTTP/1.1,200


In [66]:
type(df_ind.loc['10.131.2.1'])

pandas.core.frame.DataFrame

In [3]:
df=pd.read_csv('data\weblog.csv')
all_indicators=pd.read_csv('data/all_indicators.csv')
df['Time']=df['Time'].str.replace('[','')

print('\t\t________summary report__________\n')
print('Shape of the log Data\t\t',df.shape[0],'X',df.shape[1])
print('Data Headers\t\t\t',list(df.columns))
print('Period\t\t\t\t',df['Time'][0],'to',df['Time'][len(df)-1])
print('Number of unique Indicators\t',len(df['indicator'].unique()))
print('unique status codes\t\t',df['Status'].unique(),'\n')
unique_indicator=df['indicator'].unique()
siem_db=['70.32.39.219','91.194.254.94','128.199.199.187','69.12.73.174','94.237.37.28']

print('\t\t>>>>>>>>>>finding anamolies in logs<<<<<<<<<<<\n')
time.sleep(random.randint(7, 10))
anamolies=dict()
for indicator in unique_indicator:
    result=all_indicators[all_indicators['observable_value']==indicator]
    if len(result)>0:
        anamolies[indicator]=list(result['threat_actor'])[0]

already_available=0
not_availbale=0
already_found_anamoly=[]
not_found_anamoly=[]
for indicator in anamolies:
    if indicator in siem_db:
        already_available+=1
        already_found_anamoly.append(indicator)
    else:
        not_availbale+=1
        not_found_anamoly.append(indicator)
print('Total number of anomalies-------------',already_available+not_availbale)
print('Total number of Anomaly detected and rejected by SIEM-------------',already_available,'\n')
print(all_indicators.set_index('observable_value').loc[already_found_anamoly],'\n')
print('Total number of Anomaly not detected by SIEM-------------',not_availbale,'\n')
anamoly_groups=all_indicators.set_index('observable_value').loc[not_found_anamoly]
print(anamoly_groups)

		________summary report__________

Shape of the log Data		 15789 X 4
Data Headers			 ['indicator', 'Time', 'URL', 'Status']
Period				 29/Nov/2017:06:58:55 to 02/Mar/2018:15:47:46
Number of unique Indicators	 13
unique status codes		 [200 302 304 206 404 401] 

		>>>>>>>>>>finding anamolies in logs<<<<<<<<<<<

Total number of anomalies------------- 8
Total number of Anomaly detected and rejected by SIEM------------- 5 

                 type threat_actor
observable_value                  
70.32.39.219       ip        turla
91.194.254.94      ip     carbanak
69.12.73.174       ip        apt28
94.237.37.28       ip        apt28
128.199.199.187    ip        apt28 

Total number of Anomaly not detected by SIEM------------- 3 

                 type threat_actor
observable_value                  
81.17.30.29        ip        apt28
92.114.92.102      ip        apt28
10.74.5.100        ip     carbanak


In [4]:
def create_report(weblogfile,connection):
    
        report_data={'name':'Log_analysis_report_'+datetime.datetime.today().strftime("%Y-%m-%dT%H:%M:%S+00:00"),
                    'description':'Log analysis and anamoly detection '+str(weblogfile.split('//')[-1]),
                    'object_status':0,
                    'source_confidence_level':1,
                    'created_by_ref':'Rhymtechnologies',
                    'marking_definitions':None}
        
        report_details=pd.DataFrame([report_data.values()],columns=report_data.keys())
        for index,row in report_details.iterrows():
            report_by = connection.identity.list(filters=[{'key':'name','values': row['created_by_ref'] }])
           
            if len(report_by) == 0 : 
                warnings.warn("Since " + row['created_by_ref'] + " is not inserted in opencti.Rhymtechnologies is taken as default")
                report_by = organization = connection.identity.create(
                                            type='Organization',
                                            name="Rhymtechnologies",
                                            alias=["Rhymtech"],
                                            description="A young and innovative company formed with a vision to create world leading Cyber Security products and services. The promoters are a group of highly successful professionals with diverse backgrounds across IT Delivery, Sales and Consulting, Cyber Security, Artificial Intelligence, Machine Learning and Data Sciences. Our group of consultants led by some of the best minds in the industry have successfully provided strategic and value-add services for multiple large organizations globally.",
                                            )
            else:
                report_by = report_by[0]
            
            report = connection.report.create(
                                                            name=row['name'],
                                                            description= row['description'],
                                                            report_class="Internal Report",
                                                            published=datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00"),
                                                            createdByRef= report_by['id'],
                                                            object_status = row['object_status'],
                                                            source_confidence_level = row ['source_confidence_level'],
                                                            update = True,
                                                            )
            print("one_completed")
        return report

In [5]:
def threat_group_mapping(threat_group_name,report_id,connection,indicators):
    intrusion_set = connection.intrusion_set.read(filters=[{"key": "name", "values": [threat_group_name]}])
    stix_relations = connection.stix_relation.list(fromId=intrusion_set["id"], inferred=True)
    all_details=[]
    for relation in stix_relations:
        details=[relation['from']['id'],relation['id'],relation['to']['id'],relation['from']['name'],relation['relationship_type'],relation['to']['name'],relation['to']['entity_type']]
        all_details.append(details)
    df_relations=pd.DataFrame(all_details,columns=['from_id','relation_id','to_id','from_name','relation','to_name','to_type'])
    
    all_id=list(set(df_relations['from_id']))
    all_id.extend(set(df_relations['to_id']))
    all_id.extend(set(df_relations['relation_id']))
    
    result =list(map(lambda x:connection.report.add_stix_entity(id=report_id,entity_id =x),all_id))
    
    group_id=list(set(df_relations['from_id']))[0]
    group_id_name=connection.get_stix_domain_entity (group_id)['name'].lower()
    for indicator_id,group_name in indicators.items():
        if group_name.lower()==group_id_name:
            relation = connection.stix_relation.create(
                    fromType= connection.get_stix_domain_entity (group_id)['entity_type'],
                    fromId=group_id,
                    relationship_type ='indicates',
                    toType=opencti_api_client.get_stix_domain_entity (indicator_id)['entity_type'],
                    toId= indicator_id,
                    )
    connection.report.add_stix_entity(id=report['id'],entity_id =relation['id'])
        
    
    return df_relations

In [6]:
def create_indicator_and_add_to_report(connection,report_id,anamoly_groups):
    indicator_dict=dict()
    for index,row in anamoly_groups.iterrows():
            observables = connection.stix_observable.create(type="IPv4-Addr", observable_value=index, createIndicator=True)
            indicator_id=observables["indicators"][0]['id']
            connection.report.add_stix_entity(id=report_id,entity_id =indicator_id)
            indicator_dict[indicator_id]=row['threat_actor']
    return indicator_dict

In [7]:
api_url = "https://demo.opencti.io"
api_token = "2b4f29e3-5ea8-4890-8cf5-a76f61f1e2b2"

# OpenCTI initialization
opencti_api_client = OpenCTIApiClient(api_url, api_token)

INFO:root:Listing Threat-Actors with filters null.


In [139]:
df=pd.read_csv(sys.argv[1])
all_indicators=pd.read_csv('data/all_indicators.csv')
df['Time']=df['Time'].str.replace('[','')

In [193]:
print('\t\t________summary report__________\n')
print('Shape of the log Data\t\t',df.shape[0],'X',df.shape[1])
print('Data Headers\t\t\t',list(df.columns))
print('Period\t\t\t\t',df['Time'][0],'to',df['Time'][len(df)-1])
print('Number of unique Indicators\t',len(df['indicator'].unique()))
print('unique status codes\t\t',df['Status'].unique(),'\n')
unique_indicator=df['indicator'].unique()
siem_db=['70.32.39.219','91.194.254.94','128.199.199.187','69.12.73.174','94.237.37.28']

anamolies=dict()
for indicator in unique_indicator:
    result=all_indicators[all_indicators['observable_value']==indicator]
    if len(result)>0:
        anamolies[indicator]=list(result['threat_actor'])[0]

already_available=0
not_availbale=0
already_found_anamoly=dict()
not_found_anamoly=dict()
for indicator in anamolies:
    if indicator in siem_db:
        already_available+=1
        already_found_anamoly[indicator]=anamolies[indicator]
    else:
        not_availbale+=1
        not_found_anamoly[indicator]=anamolies[indicator]
print('Total number of anomalies-------------',already_available+not_availbale)
print('Total number of Anomaly detected and rejected by SIEM-------------',already_available,'\n')
print(all_indicators.set_index('observable_value').loc[already_found_anamoly.keys()])
print('Total number of Anomaly not detected-------------',not_availbale,'\n')
print(all_indicators.set_index('observable_value').loc[not_found_anamoly.keys()])

		________summary report__________

Shape of the log Data		 15789 X 4
Data Headers			 ['indicator', 'Time', 'URL', 'Status']
Period				 29/Nov/2017:06:58:55 to 02/Mar/2018:15:47:46
Number of unique Indicators	 13
unique status codes		 [200 302 304 206 404 401] 

Total number of anomalies------------- 8
Total number of Anomaly detected and rejected by SIEM------------- 5 

                 type threat_actor
observable_value                  
70.32.39.219       ip        turla
91.194.254.94      ip     carbanak
69.12.73.174       ip        apt28
94.237.37.28       ip        apt28
128.199.199.187    ip        apt28
Total number of Anomaly not detected------------- 3 

                 type threat_actor
observable_value                  
81.17.30.29        ip        apt28
92.114.92.102      ip        apt28
10.74.5.100        ip     carbanak


In [173]:
a=[('70.32.39.219', 'turla'), ('91.194.254.94', 'carbanak'), ('69.12.73.174', 'apt28'), ('94.237.37.28', 'apt28'), ('128.199.199.187', 'apt28')]

In [184]:
not_found_anamoly.keys()

dict_keys(['81.17.30.29', '92.114.92.102', '10.74.5.100'])

In [190]:
all_indicators.set_index('observable_value').loc[['81.17.30.29', '92.114.92.102', '10.74.5.100']]

,type,threat_actor
observable_value,,
81.17.30.29,ip,apt28
92.114.92.102,ip,apt28
10.74.5.100,ip,carbanak


In [97]:
import os
initial=True
path=r'D:\MY STUFF\rhym\data\mitre_data_automation\ISR\automated_collected_mitre_data'
for group in os.listdir(path):
    indicators=pd.read_csv(path+os.sep+group+os.sep+'indicators.csv')
    indicators.dropna(inplace=True)
    if len(indicators)>0:
        if initial:
            indicators['threat_actor']=group
            all_indicators=indicators
            initial=False
        else:
            indicators['threat_actor']=group
            all_indicators=all_indicators.append(indicators)        

In [99]:
all_indicators.to_csv(r'D:\MY STUFF\rhym\data\mitre_data_automation\ISR\all_indicators.csv',index=None)